In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
# Required libraries
# We will try several Machine Learning platforms
from __future__ import print_function
from builtins import str
from builtins import range

import os
import sys
import tarfile

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from io import BytesIO

import bson
import json 
import skimage

import matplotlib.pyplot as plt

from sklearn import *
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import time
#import datetime as dt
from datetime import datetime as dt
import multiprocessing
import psutil

import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score

# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline


from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))
print("pandas --version : ", pd.__version__)
print("python --version : ", sys.version)
PyVersion = sys.version
print(os.listdir("../input"))

### 1- Data loading and Data Engineering
Let us reduce the data size in memory

In [ ]:
# From https://www.kaggle.com/theoviel/load-the-totality-of-the-data 
# https://www.kaggle.com/xhlulu/load-entire-dataset-with-7-gb-ram-fork
dtypes = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float16',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int8',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float32',
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float16',
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float16',
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float32',
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float32',
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float16',
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float16',
        'Census_InternalPrimaryDisplayResolutionVertical':      'float16',
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float32',
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }

In [ ]:
train_df = pd.read_csv('../input/train.csv', nrows = 200, dtype=dtypes)
columns = train_df.columns
train_df = pd.read_csv('../input/train.csv', nrows = 2000000, usecols= columns, dtype=dtypes)
#train_df.info() 

In [ ]:
numerics = ['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64'] 
numerical_columns = [c for c,v in dtypes.items() if v in numerics] 
categorical_columns = [c for c,v in dtypes.items() if v not in numerics] 

In [ ]:
train_df.shape

In [ ]:
train_df.to_pickle("train_df.pkl")

In [ ]:
# MEAN ENCODINGS
for elt in numerical_columns:
    if elt != 'HasDetections':
        m_id = elt 
        target = 'HasDetections' 
        new_column = pd.DataFrame(columns=[m_id+'_mean_target']) 
        # mean encoding of variables
        train_df[m_id+'_mean_target'] = new_column[m_id+'_mean_target'] 
        cumsum = train_df.groupby(m_id)[target].cumsum() - train_df[target] 
        cumcnt = train_df.groupby(m_id).cumcount() 
        train_df[m_id+'_mean_target'] = cumsum/cumcnt
    
train_df.shape

### 2- Data splitting

In [ ]:
labels_df = train_df['HasDetections']
train_df = train_df.drop(['MachineIdentifier'], axis=1) 
columns = train_df.columns 
train_df.shape

In [ ]:
train_df.head(2)

In [ ]:
col = list(columns)
col.remove('HasDetections')
#print(col), len(col)
len(col)

In [ ]:
# DATA SPLITING
X_train, X_test, y_train, y_test = train_test_split(train_df[col], labels_df, test_size=0.2, 
                                                    random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, 
                                                    random_state=42)

n_features = X_train.shape[1]
n_classes = len(np.unique(y_train))
print("n_features : {}\nn_classes : {}\nX_train.shape : {}".format(n_features, n_classes, 
                                                                   X_train.shape))

In [ ]:
X_train.shape, X_val.shape, X_test.shape

In [ ]:
y_train.shape, y_val.shape, y_test.shape

### 3- First training & Hyperparameter Optimization

In [ ]:
print('Start training...')
# train
gbm = lgb.LGBMClassifier(objective='binary',
                        num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=20)
gbm.fit(X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='auc',
        early_stopping_rounds=100)

print('Start predicting...')
# predict
#y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration_)
y_pred = gbm.predict(X_test, num_iteration=10)
# eval
print('The log_loss of prediction is:', log_loss(y_test, y_pred))
print('The roc_auc_score of prediction is:', roc_auc_score(y_test, y_pred))

# feature importances
print('\nNumber of features :', len(list(gbm.feature_importances_)))
print('Features :', train_df.columns)
print('Importances :', list(gbm.feature_importances_))
print('\nFeature importances :', dict(zip(train_df.columns,list(gbm.feature_importances_))))

# The log_loss of prediction is: 12.769218206519513 , roc_auc_score is: 0.630302822037965
# for test size == 20% and val size == 20%
# 0.630302822037965
# log_loss: 12.587463101120566 et roc_auc_score: 0.6355364487531907 for mean encoding with dummify.
# log_loss: 12.494988259947414 et roc_auc_score: 0.638198925649798 for mean encoding without dummify.

#### <font color='red'>Please, unlock this part *(transform into code cell)* for fine-tuning phase.</font>
### ### Hyperparameter Optimization ##############
#### other scikit-learn modules
estimator = lgb.LGBMClassifier(num_leaves=31)

#### The parameters used are in comment below, it will take too long time to run them here
param_grid = {
    'learning_rate': [0.1],
    'n_estimators': [100, 500],
    'num_leaves': [20, 31],
    'min_data_in_leaf': [5, 10],
    'reg_alpha': [0],
    'reg_lambda': [1e-6], 
    'bagging_fraction': [0.8, 0.9],
    'min_child_samples': [10, 20],
    'min_child_weight': [1e-6], 
    'max_bin': [256]
}

gbm = GridSearchCV(estimator, param_grid, cv=5)

gbm.fit(X_train, y_train)

print('\n\nBest parameters found by grid search are:', gbm.best_params_)

'''
param_grid = {
    'learning_rate': [0.01, 0.1, 0.05, 0.07, 1],
    'n_estimators': [20, 40, 100, 500],
    'num_leaves': [20, 31, 50, 127],
    'min_data_in_leaf': [5, 10, 20, 50, 100],
    'reg_alpha': [0, 1e-3, 1e-6],
    'reg_lambda': [0, 1e-3, 1e-6], 
    'bagging_fraction': [0.5, 0.6, 0.7, 0.8, 0.9],
    'min_child_samples': [10, 20, 30],
    'min_child_weight': [5, 1e-3, 1e-6], 
    'max_bin': [255, 256]
}
'''
####

### 4- Fine Tuning & Evaluation

In [ ]:
# FEATURES TUNING IF NECESSARY
## For example, we could remove least important features if required
## And also use the best parameters provided by the grid search Cross Validation
# Here I reuse the same previous splits instead of recreate a new one.
X_train = pd.DataFrame(X_train, columns=col) 
X_val = pd.DataFrame(X_val, columns=col) 

In [ ]:
X_train.head(2)

In [ ]:
X_val.head(2)

In [ ]:
# In case I decide to delete the features with lower importance to see how that could improve the result
# Using the Feature importances Dictionary
feature_importances_dic = dict(zip(train_df.columns,list(gbm.feature_importances_)))
banned_columns = []
for key in feature_importances_dic:
    if feature_importances_dic[key]==0:
        banned_columns.append(key)
print("len banned_columns:", len(banned_columns))
print("banned_columns:\n", banned_columns)

new_cols = [c for c in train_df.columns if c not in banned_columns]
new_cols = list(new_cols)
if 'HasDetections' in new_cols:
    new_cols.remove('HasDetections')
print("new columns:\n", new_cols) 
len(new_cols)

In [ ]:
X_train_new = X_train[new_cols] 
X_val_new = X_val[new_cols] 
X_test_new = X_test[new_cols] 
X_train_new.shape, X_val_new.shape, X_test_new.shape 

In [ ]:
# TRAINING
# specify your configurations as a dict
# Use the best parameters provided by the grid search Cross Validation
params = {"objective": "binary",
          #"sigmoid":1.0,
          "task": "train",
          "boosting_type": "gbdt",
          "learning_rate": 0.05,
          "num_leaves": 31, # 31, 20
          "max_bin": 256,
          "min_data_in_leaf": 5, # Problem  2000
          "feature_fraction": 0.6, # 0.6
          "verbosity": 0,
          "seed": 0,
          "drop_rate": 0.1, # 0.1
          "is_unbalance": False,
          "max_drop": 50,
          "min_child_samples": 10,
          "min_child_weight": 1e-06, # 5
          "min_split_gain": 0,
          "colsample_bytree": 0.6343275033,
          "max_depth": 8, # 8
          #"n_estimators": 500, # 500
          "nthread": -1,
          "reg_alpha": 0,
          "reg_lambda": 1e-06,# 1
          #"silent": True,
          "subsample_for_bin": 50000, # 50000
          "subsample_freq": 1, # 1
          #"min_data":1,
          #"min_data_in_bin":1,
          'metric': {'binary_logloss'},
          'bagging_fraction': 0.8,
          'bagging_freq': 5,
          #'num_iterations':1000,
          "subsample": 0.733
          }

In [ ]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

#lgb_train = lgb.Dataset(X_train_new, y_train)
#lgb_eval = lgb.Dataset(X_val_new, y_val, reference=lgb_train)


print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=2000,
                valid_sets=lgb_eval,
                early_stopping_rounds=120)

print('Save model...')
# save model to file
gbm.save_model('model.txt')

In [ ]:
print('Start predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
#y_pred = gbm.predict(X_test_new, num_iteration=gbm.best_iteration)
# eval
#print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)
print('The log_loss of prediction is:', log_loss(y_test, y_pred))
print('The roc_auc_score of prediction is:', roc_auc_score(y_test, y_pred))
# The log_loss of prediction is: 0.6147321812202021 and the roc_auc_score is: 0.7143388814690966
# for test size == 10% and val size == 20%
# With dummification, log_loss: 0.6103008030367194 and roc_auc_score: 0.7168178922141861
# Without dummification, log_loss: 0.6067172451965305 and roc_auc_score: 0.7217849368889165

In [ ]:
print('Loading model to predict...')
# load model to predict
bst = lgb.Booster(model_file='model.txt')
# can only predict with the best iteration (or the saving iteration)
y_pred = bst.predict(X_test)
print('The log_loss of prediction is:', log_loss(y_test, y_pred))
print('The roc_auc_score of prediction is:', roc_auc_score(y_test, y_pred))

### 5- SUBMISSION FILE CREATION

In [ ]:
gbm.best_iteration, bst.best_iteration

In [ ]:
!ls

In [ ]:
# We will now save the parameters in order to reuse them in another kernel for submission
del train_df
train_df = pd.read_pickle("train_df.pkl")
train_df.shape

In [ ]:
train_df2 = pd.read_csv('../input/train.csv', nrows = 200, dtype=dtypes)
columns = train_df2.columns
columns = list(columns)
columns.remove('HasDetections')
test_df = pd.read_csv('../input/test.csv', nrows = 2000, usecols= columns, dtype=dtypes)

for elt in numerical_columns: 
    if elt != 'HasDetections': 
        m_id = elt 
        target = 'HasDetections' 
        new_column = pd.DataFrame(columns=[m_id+'_mean_target']) 
        # mean encoding of variables
        test_df[m_id+'_mean_target'] = new_column[m_id+'_mean_target'] 
        cumsum = train_df.groupby(m_id)[target].cumsum() - train_df[target] 
        cumcnt = train_df.groupby(m_id).cumcount() 
        test_df[m_id+'_mean_target'] = cumsum/cumcnt
        
test_df.shape

In [ ]:
xtest = test_df[col] 
preds = gbm.predict(xtest, num_iteration=gbm.best_iteration) # data_has_header=False 

In [ ]:
len(preds) 

In [ ]:
test_df.shape

In [ ]:
sub_sample_df = pd.read_csv('../input/sample_submission.csv')
len(sub_sample_df)

In [ ]:
sub_sample_df.head(2)

In [ ]:
if len(sub_sample_df) == len(preds):
    submission = pd.DataFrame({'MachineIdentifier': sub_sample_df["MachineIdentifier"], 
                               'HasDetections': preds})
    submission.head(10)
else:
    print("The prediction size must be: {}, but we got: {}".format(len(sub_sample_df), len(preds)))

In [ ]:
#submission.to_csv("Micro_MP_LGBMsubmission.csv", index=False)

In [ ]:
del test_df 

#### In order to reuse the saved variables for test prediction and submission, in case there is not enough memory space on Kaggle

In [ ]:
# np.savez Cannot handle categorical data, I have to try pandas to_pickle.

In [ ]:
#### Backup the 'model.txt' file, the train_df. 
del X_train
X_test.to_pickle("X_test.pkl")
X_test_new.to_pickle("X_test_new.pkl")
y_test.to_pickle("y_test.pkl") 

In [ ]:
del train_df, X_train_new, X_val_new, X_val, X_test_new, X_test

In [ ]:
!ls

In [ ]:
#### For reusing 
X_test = pd.read_pickle("X_test.pkl")
X_test.shape

In [ ]:
!tar cvf 'model_backup.tar' 'model.txt' 

In [ ]:
import shutil 
shutil.make_archive('model_backup', 'zip', '.', 'model.txt') 

In [ ]:
!ls